# Programación avanzada- UNICABA

## Ejercicio de cierre de unidad 2

### Analisis de trends musicales en streaming segun Billboard

Tipo: Texto (string)

In [2]:
######################-DICCIONARIO DE DATOS DEL DATASET-###########

# 1-Date: Tipo: Fecha (formato yyyy/mm/dd) #    Descripción: Fecha en la que se registró el ranking de la canción.
# 2-Song: Tipo: Texto (string) #    Descripción: Nombre de la canción en el ranking.
# 3-Artist: Tipo: Texto (string) #    Descripción: Nombre del artista o grupo musical (puede incluir “Featuring” si hay colaboraciones).
# 4-Rank: Tipo: Numérico entero #    Descripción: Posición actual de la canción en el ranking (1 = primer lugar).
# 5-Last_Week:Tipo: Numérico entero #    Descripción: Posición que ocupaba la canción en la semana anterior.
# 6-Peak_Position:Tipo: Numérico entero #    Descripción: Mejor posición alcanzada por la canción en el ranking hasta la fecha.
# 7-Weeks_in_Charts:Tipo: Numérico entero #    Descripción: Número de semanas que la canción lleva en el ranking.

In [3]:
import os
from supabase import create_client, Client
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [4]:
# Configuración
SUPABASE_URL = os.environ.get("SUPABASE_URL")
SUPABASE_KEY = os.environ.get("SUPABASE_KEY")
TABLE_NAME = "Dataset_Ranking"
PAGE_SIZE = 1000  # Límite por defecto de Supabase
# ---------------------------------------------
# Inicialización
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)
all_records = []
offset = 0

print(f"Iniciando descarga paginada de la tabla '{TABLE_NAME}'...")

while True:
    #  Definir el rango de filas a solicitar (paginación)
    start_row = offset
    end_row = offset + PAGE_SIZE - 1

    # print(f" -> Fetching filas {start_row} a {end_row}...")

    #  Ejecutar la consulta con el rango
    response = (
        supabase.table(TABLE_NAME)
        .select("*")
        .order("Date", desc=True)
        .range(start_row, end_row)  #  La clave de la paginación
        .execute()
    )

    data = response.data

    #  Romper el bucle si no hay más datos
    if not data:
        print("Descarga completa: No se encontraron más registros.")
        break

    #  Acumular los registros
    all_records.extend(data)

    #  Aumentar el desplazamiento para la siguiente página
    offset += PAGE_SIZE

#  Crear el DataFrame ÚNICO
if all_records:
    df = pd.DataFrame(all_records)
    print(f"\n  {len(df)} registros totales cargados en un solo DataFrame.")

else:
    print("La tabla está vacía.")

Iniciando descarga paginada de la tabla 'Dataset_Ranking'...
Descarga completa: No se encontraron más registros.

  33050 registros totales cargados en un solo DataFrame.
Descarga completa: No se encontraron más registros.

  33050 registros totales cargados en un solo DataFrame.


In [5]:
# dataframe
# df = pd.read_csv("/work/streaming_songs.csv")
df.head(10)

,ID,Date,Song,Artist,Rank,Last_Week,Peak_Position,Weeks_in_Charts
0,14772,2025-09-24,Just Keep Watching,Tate McRae,44,17,7,0
1,24330,2025-09-24,Birds Of A Feather,Billie Eilish,37,41,2,70
2,16036,2025-09-24,Dreams,Fleetwood Mac,47,43,6,35
3,17423,2025-09-24,How It's Done,HUNTR/X: EJAE| Audrey Nuna & REI AMI,5,5,5,12
4,12230,2025-09-24,Love Me Not,Ravyn Lenae,23,24,10,20
5,6562,2025-09-24,It Depends,Chris Brown Featuring Bryson Tiller,50,49,19,8
6,8657,2025-09-24,House Tour,Sabrina Carpenter,32,21,15,3
7,3082,2025-09-24,No Broke Boys,Disco Lines & Tinashe,29,44,29,5
8,33047,2025-09-24,Die With A Smile,Lady Gaga & Bruno Mars,30,36,1,57
9,18923,2025-09-24,I Got Better,Morgan Wallen,11,12,4,18


In [6]:
# encontrar vacios
df.isnull().sum()
df.info()
print(f"Filas: {df.shape[0]}, Columnas: {df.shape[1]}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33050 entries, 0 to 33049
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               33050 non-null  int64 
 1   Date             33050 non-null  object
 2   Song             33050 non-null  object
 3   Artist           33050 non-null  object
 4   Rank             33050 non-null  int64 
 5   Last_Week        33050 non-null  int64 
 6   Peak_Position    33050 non-null  int64 
 7   Weeks_in_Charts  33050 non-null  object
dtypes: int64(4), object(4)
memory usage: 2.0+ MB
Filas: 33050, Columnas: 8


In [7]:
df.describe()

,ID,Rank,Last_Week,Peak_Position
count,33050.000000,33050.000000,33050.000000,33050.000000
mean,16525.500000,25.500000,23.190439,10.174554
std,9540.857535,14.431088,14.032791,11.959740
min,1.000000,1.000000,1.000000,1.000000
25%,8263.250000,13.000000,11.000000,1.000000
50%,16525.500000,25.500000,22.000000,5.000000
75%,24787.750000,38.000000,35.000000,15.000000
max,33050.000000,50.000000,50.000000,148.000000


Los datos fueron recolectados desde el 23/01/2013 hasta el 24/09/2025, contiene 33050 registros de canciones de streaming calculadas por Billboard

In [8]:
# columna Date a datetime
df["Date"] = pd.to_datetime(df["Date"], format="%Y-%m-%d", errors="coerce")

# Extraer el año
df["Year"] = df["Date"].dt.year

# canciones por año

songs_per_year = df.groupby("Year")["Song"].nunique().reset_index()
songs_per_year.columns = ["Year", "UniqueSongCount"]

average_unique_songs_per_year = songs_per_year["UniqueSongCount"].mean()
print(f"Promedio de canciones únicas por año: {average_unique_songs_per_year:.2f}")

Promedio de canciones únicas por año: 363.46


In [9]:
# Graficar
fig = px.line(
    songs_per_year,
    x="Year",
    y="UniqueSongCount",
    markers=True,
    title="Evolución de canciones por año",
    labels={"Year": "Año", "UniqueSongCount": "Cantidad de canciones"},
)

fig.update_layout(template="plotly_white")
fig.show()

Hay una media de 343 canciones por año, se evidencia una mayor cantidad de canciones en streaming a medida que pasa el tiempo. Nota: la cantidad en el 2025 se debe a que solo se incluye hasta septiembre

In [10]:
# canciones únicas
unique_songs = df["Song"].nunique()

#  card
fig = go.Figure(
    go.Indicator(
        mode="number",
        value=unique_songs,
        title={"text": " Canciones únicas en streaming"},
        number={"valueformat": ","},
    )
)

fig.update_layout(height=200, margin=dict(t=30, b=0, l=0, r=0), template="plotly_white")

fig.show()

In [11]:
#  artistas únicos
unique_artists = df["Artist"].nunique()

#  card
fig = go.Figure(
    go.Indicator(
        mode="number",
        value=unique_artists,
        title={"text": " Artistas únicos"},
        number={"valueformat": ","},
    )
)

fig.update_layout(height=200, margin=dict(t=30, b=0, l=0, r=0), template="plotly_white")

fig.show()

In [12]:
# Filtrar solo las filas donde Peak Position es 1
top_hits = df[df["Peak_Position"] == 1]

#  #1 de cada artista
artist_counts = top_hits["Artist"].value_counts()

# artista con más #1
top_artist = artist_counts.idxmax()
top_count = artist_counts.max()
print(f"El artista con más semanas en el puesto #1 es: {top_artist} ({top_count})")
print(artist_counts.head(10))

El artista con más semanas en el puesto #1 es: Taylor Swift (348)
Artist
Taylor Swift              348
Drake                     303
Morgan Wallen             231
The Weeknd                170
Kendrick Lamar            166
Ariana Grande             155
Miley Cyrus               148
PSY                       123
Olivia Rodrigo            119
Post Malone & Swae Lee    117
Name: count, dtype: int64


In [13]:
# top 10 artistas
top_10_artists = artist_counts.head(10).index.tolist()

# Filtrar los datos de esos artistas
top_10_data = top_hits[top_hits["Artist"].isin(top_10_artists)]

# Agrupar por año y artista
artist_year_counts = (
    top_10_data.groupby(["Year", "Artist"]).size().reset_index(name="NumberOneHits")
)
fig = px.bar(
    artist_year_counts,
    x="Year",
    y="NumberOneHits",
    color="Artist",
    barmode="group",
    title=" Evolución anual de los top 10 artistas con canciones en el puesto #1",
    labels={"Year": "Año", "NumberOneHits": "#1 por año"},
)

fig.update_layout(
    legend=dict(orientation="h", yanchor="bottom", y=-0.3, xanchor="center", x=0.5),
    template="plotly_white",
    height=600,
)

fig.show()

In [14]:
#  top 10 artistas
top_10_artists = artist_counts.head(5).index.tolist()

# Filtrar solo los datos de esos artistas
top_10_data = top_hits[top_hits["Artist"].isin(top_10_artists)]

# Agrupar por año y artista
artist_year_counts = (
    top_10_data.groupby(["Year", "Artist"]).size().reset_index(name="NumberOneHits")
)
import plotly.express as px

fig = px.line(
    artist_year_counts,
    x="Year",
    y="NumberOneHits",
    color="Artist",
    markers=True,
    title=" Evolución temporal de los top 10 artistas con canciones en el puesto #1",
    labels={"Year": "Año", "NumberOneHits": "#1 por año"},
)

fig.update_layout(
    legend=dict(orientation="h", yanchor="bottom", y=-0.3, xanchor="center", x=0.5),
    template="plotly_white",
    height=600,
)

fig.show()

In [15]:
top_1 = df[df["Peak_Position"] == 1].copy()
# Extraer el año
top_1["year"] = top_1["Date"].dt.year

# Contar artistas únicos por año
artistas_por_anio = top_1.groupby("Year")["Artist"].nunique().reset_index()
artistas_por_anio.columns = ["año", "artistas_unicos_en_posicion_1"]

print(artistas_por_anio)

fig = px.line(
    artistas_por_anio,
    x="año",
    y="artistas_unicos_en_posicion_1",
    markers=True,
    title="Artistas únicos en la posición #1 por año",
    labels={"anio": "Año", "artistas_unicos_en_posicion_1": "Artistas únicos en #1"},
)

fig.update_layout(
    xaxis=dict(dtick=1), yaxis=dict(title="Cantidad de artistas"), hovermode="x unified"
)

fig.show()

     año  artistas_unicos_en_posicion_1
0   2013                            182
1   2014                            140
2   2015                            124
3   2016                            144
4   2017                            166
5   2018                            200
6   2019                            198
7   2020                            253
8   2021                            239
9   2022                            229
10  2023                            197
11  2024                            199
12  2025                            132


In [16]:
# mejor posición que alcanzó cada canción
best_positions = df.groupby("Song", as_index=False)["Peak_Position"].min()

# Unir con los artistas
song_artist = df.groupby("Song", as_index=False)["Artist"].first()

# Merge para tener artista + mejor posición
merged = pd.merge(best_positions, song_artist, on="Song")

# Filtrar canciones que llegaron al top 10
top_10_songs = merged[merged["Peak_Position"] <= 10]

#  artistas únicos que llegaron al top 10
top_10_artists = top_10_songs["Artist"].unique()

# Contar cuántos artistas son
num_top_10_artists = len(top_10_artists)

print(f" Total de artistas que llegaron alguna vez al top 10: {num_top_10_artists}")

 Total de artistas que llegaron alguna vez al top 10: 1700


In [17]:
# Revisar el error y corregir el código
# Parece que hay un error en la creación del DataFrame, ya que se está utilizando el índice en lugar de los valores de la columna "Artist".

# Corregir el código
# Crear un DataFrame con los 10 artistas con más semanas en la última posición
last_position_hits = df[df["Rank"] == 50]

# Contar el número de semanas que cada artista ha estado en la última posición
last_position_counts = last_position_hits["Artist"].value_counts().head(10)

# Crear un DataFrame para los 10 artistas con más semanas en la última posición
top_10_last_position = last_position_counts.reset_index()
top_10_last_position.columns = ["Artist", "Weeks in Last Position"]

# Graficar
fig = px.bar(
    top_10_last_position,
    x="Weeks in Last Position",
    y="Artist",
    orientation="h",
    title="Top 10 Artistas con más semanas en la última posición",
    text="Weeks in Last Position",
)

fig.update_traces(texttemplate="%{text}", textposition="inside")
fig.update_layout(
    xaxis_title="Semanas en la última posición",
    yaxis_title="Artista",
    template="plotly_white",
    height=600,
)

fig.show()

### Tendencia de cantidad de artistas nuevos por año

In [53]:
# Tendencia de cantidad de artistas nuevos por año
import plotly.express as px
df["Año"] = df["Date"].dt.year
# Encontrar el primer año en que cada artista aparece
primer_anio_artista = df.groupby("Artist")["Año"].min().reset_index()
artistas_nuevos_por_anio = primer_anio_artista.groupby("Año").size().reset_index(name="Artistas nuevos")
fig = px.line(artistas_nuevos_por_anio, x="Año", y="Artistas nuevos", title="Tendencia de artistas nuevos por año", labels={"Año": "Año", "Artistas nuevos": "Cantidad de artistas"})
fig.update_layout(template="plotly_white", xaxis_title="Año", yaxis_title="Cantidad de artistas nuevos")
fig.show()

La tendencia revela ciclos de auge y caída en la incorporación de artistas nuevos, con un crecimiento importante de artistas entre 2016 y 2020, pero una disminución notoria en los años más recientes.

### Top 10 canciones de Taylor Swift que alcanzaron más veces la posición #1

In [50]:
# Top 10 canciones de Taylor Swift que alcanzaron más veces la posición #1
import plotly.express as px

# Filtrar canciones de Taylor Swift
taylor_df = df[df["Artist"].fillna("").str.contains("Taylor Swift", case=False, na=False)]

# Filtrar solo las filas donde Peak_Position es 1
taylor_top1 = taylor_df[taylor_df["Peak_Position"] == 1]

# Contar cuántas veces cada canción alcanzó la posición 1
canciones_top1 = taylor_top1["Song"].value_counts().reset_index()
canciones_top1.columns = ["Song", "Veces_en_posicion_1"]

# Seleccionar el top 10
top_10_canciones = canciones_top1.head(10)

if top_10_canciones.empty:
    print("No hay suficientes datos para mostrar el top 10 de canciones de Taylor Swift en la posición #1.")
else:
    fig = px.bar(
        top_10_canciones,
        x="Song",
        y="Veces_en_posicion_1",
        color="Song",
        title="Top 10 canciones de Taylor Swift que alcanzaron más veces la posición #1",
        labels={"Veces_en_posicion_1": "Veces en posición #1", "Song": "Canción"}
    )
    fig.update_traces(texttemplate="%{y}", textposition="outside", showlegend=False)
    fig.update_layout(template="plotly_white", xaxis_title="Canción", yaxis_title="Veces en posición #1", height=700, margin=dict(t=60, b=60, l=80, r=80), showlegend=False)
    fig.show()

El gráfico muestra que Taylor Swift ha logrado consolidar múltiples éxitos en el puesto #1, destacando “Anti-Hero” y “Shake It Off” como sus mayores hits. Aunque algunas canciones tienen menor presencia, la tendencia refleja su capacidad constante para dominar los rankings a lo largo de distintas etapas de su carrera.

### Tendencia de cantidad de canciones nuevas por año del artista Drake

In [51]:
# Tendencia de cantidad de canciones nuevas por año del artista Drake
import plotly.express as px

# Filtrar solo las canciones de Drake
drake_df = df[df["Artist"].fillna("").str.contains("Drake", case=False, na=False)].copy()

# Extraer el año de la columna Date
drake_df["Año"] = drake_df["Date"].dt.year

# Encontrar el primer año en que cada canción de Drake aparece
primer_anio_cancion = drake_df.groupby("Song")["Año"].min().reset_index()

# Contar cuántas canciones nuevas lanzó Drake por año
canciones_nuevas_por_anio = primer_anio_cancion.groupby("Año").size().reset_index(name="Canciones nuevas")

fig = px.line(
    canciones_nuevas_por_anio,
    x="Año",
    y="Canciones nuevas",
    markers=True,
    title="Tendencia de cantidad de canciones nuevas por año de Drake",
    labels={"Año": "Año", "Canciones nuevas": "Canciones nuevas"}
)
fig.update_layout(template="plotly_white", xaxis_title="Año", yaxis_title="Canciones nuevas", height=500)
fig.show()

El gráfico refleja la tendencia de nuevas canciones de Drake a lo largo de los años. Se observa una fuerte variabilidad: con picos destacados en 2018 y entre 2021-2022, donde lanzó la mayor cantidad de temas, y caídas notables en 2014 y 2024. En general, la producción musical de Drake muestra ciclos de alta intensidad seguidos de descensos, lo que sugiere etapas de gran actividad creativa alternadas con periodos de menor lanzamiento.

### Mapa de calor: Top 10 artistas con más semanas en el ranking mostrando cuántas veces alcanzaron el puesto #1 por año

In [52]:
# Mapa de calor: Top 10 artistas con más semanas en el ranking mostrando cuántas veces alcanzaron el puesto #1 por año
import plotly.express as px

# Filtrar solo las filas donde Peak_Position es 1 (puesto #1)
top_hits = df[df["Peak_Position"] == 1].copy()

# Contar semanas en el ranking por artista
artist_weeks = df.groupby("Artist")["Weeks_in_Charts"].sum().sort_values(ascending=False)

# Seleccionar los 10 artistas con más semanas en el ranking
top_10_artists = artist_weeks.head(10).index.tolist()

# Filtrar los datos de esos artistas en el puesto #1
top_10_data = top_hits[top_hits["Artist"].isin(top_10_artists)]

# Extraer el año
top_10_data["Año"] = top_10_data["Date"].dt.year

# Agrupar por año y artista, contar cuántas veces alcanzaron el puesto #1
heatmap_data = top_10_data.groupby(["Año", "Artist"]).size().reset_index(name="Veces_en_posicion_1")

# Crear el mapa de calor
fig = px.density_heatmap(
    heatmap_data,
    x="Año",
    y="Artist",
    z="Veces_en_posicion_1",
    color_continuous_scale="Viridis",
    title="Mapa de calor: Top 10 artistas con más semanas en el ranking en el puesto #1 por año",
    labels={"Año": "Año", "Artist": "Artista", "Veces_en_posicion_1": "#1 por año"}
)
fig.update_layout(height=600, template="plotly_white")
fig.show()

El mapa refleja cómo distintos artistas han dominado en años específicos, sin una continuidad prolongada en el tiempo, lo que sugiere que el liderazgo en el ranking cambia de forma constante entre diferentes géneros y colaboraciones.